In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path

# Locate and add the package root
cwd = Path.cwd().resolve()
for parent in [cwd, *cwd.parents]:
    if (parent / "sociopathit").exists():
        ROOT = parent
        break
else:
    raise FileNotFoundError("Could not locate the sociopathit package root.")

# Add to sys.path for imports
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print(f"Added to sys.path: {ROOT}")

Added to sys.path: C:\Users\alecw\OneDrive - University of Toronto\Directives\GITTYSBURG\sociopathit


# Regression Models

In [2]:
import importlib
from sociopathit.analyses import regress as regress_module

importlib.reload(regress_module)
from sociopathit.analyses.regress import ols, logit, poisson, RegressionModel, compare_models

# Simulate data for regression
np.random.seed(42)
n = 200
df_regress = pd.DataFrame({
    'age': np.random.normal(45, 15, n),
    'income': np.random.normal(50000, 20000, n),
    'education': np.random.choice([12, 14, 16, 18], n),
    'satisfaction': np.random.normal(7, 2, n),
    'employed': np.random.choice([0, 1], n, p=[0.2, 0.8]),
    'count_events': np.random.poisson(3, n),
    'weight': np.random.uniform(0.5, 1.5, n)
})

# Make satisfaction related to predictors
df_regress['satisfaction'] = (
    5 + 
    0.03 * df_regress['age'] + 
    0.00002 * df_regress['income'] + 
    0.2 * df_regress['education'] +
    np.random.normal(0, 1, n)
)

print(df_regress.head())
print(f"\nData shape: {df_regress.shape}")

         age        income  education  satisfaction  employed  count_events  \
0  52.450712  57155.747207         16     10.270672         1             2   
1  42.926035  61215.690527         16      9.912903         1             1   
2  54.715328  71661.024864         12      9.991937         1             0   
3  67.845448  71076.041041         12      9.903556         1             4   
4  41.487699  22446.612641         18     10.416234         1             3   

     weight  
0  1.484670  
1  1.437388  
2  0.543174  
3  0.664815  
4  0.631729  

Data shape: (200, 7)


In [3]:
# Test 1: OLS Regression
print("=" * 60)
print("TEST 1: OLS Regression")
print("=" * 60)

model1 = ols(
    df=df_regress,
    outcome='satisfaction',
    inputs=['age', 'income', 'education'],
    robust=True
)

print("\nModel Summary:")
print(model1.summary())

print("\nTidy Results:")
print(model1.get_tidy())

print("\nModel Statistics:")
print(model1.get_stats())

TEST 1: OLS Regression

Model Summary:
                            OLS Regression Results                            
Dep. Variable:           satisfaction   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     43.58
Date:                Wed, 15 Oct 2025   Prob (F-statistic):           1.27e-21
Time:                        12:12:45   Log-Likelihood:                -272.29
No. Observations:                 200   AIC:                             552.6
Df Residuals:                     196   BIC:                             565.8
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        

In [4]:
# Test 2: Weighted OLS Regression
print("=" * 60)
print("TEST 2: Weighted OLS Regression")
print("=" * 60)

model2 = ols(
    df=df_regress,
    outcome='satisfaction',
    inputs=['age', 'income', 'education'],
    weight='weight',
    robust=True
)

print("\nTidy Results (Weighted):")
print(model2.get_tidy())

TEST 2: Weighted OLS Regression

Tidy Results (Weighted):
        term  estimate  std.error  statistic       p.value  conf.low  \
0      const  5.165550   0.550893   9.376689  6.807718e-21  4.085820   
1        age  0.027861   0.005502   5.063826  4.109257e-07  0.017078   
2     income  0.000015   0.000004   3.976676  6.988537e-05  0.000008   
3  education  0.219408   0.033506   6.548376  5.816620e-11  0.153738   

   conf.high  
0   6.245280  
1   0.038645  
2   0.000022  
3   0.285078  


In [5]:
# Test 3: Logistic Regression
print("=" * 60)
print("TEST 3: Logistic Regression")
print("=" * 60)

model3 = logit(
    df=df_regress,
    outcome='employed',
    inputs=['age', 'education'],
    robust=True
)

print("\nLogistic Regression Results:")
print(model3.get_tidy())

print("\nModel Statistics:")
print(model3.get_stats())

TEST 3: Logistic Regression
Optimization terminated successfully.
         Current function value: 0.495807
         Iterations 5

Logistic Regression Results:
        term  estimate  std.error  statistic   p.value  conf.low  conf.high
0      const  0.949878   1.411842   0.672793  0.501079 -1.817283   3.717038
1        age -0.013852   0.012969  -1.068082  0.285483 -0.039271   0.011567
2  education  0.072462   0.086165   0.840969  0.400366 -0.096418   0.241342

Model Statistics:
{'N': 200, 'AIC': np.float64(204.3227910665525), 'BIC': np.float64(214.2177431661966), 'Log-Likelihood': np.float64(-99.16139553327625), 'Pseudo_R_squared': np.float64(0.009183500482258689)}


In [6]:
# Test 4: Poisson Regression
print("=" * 60)
print("TEST 4: Poisson Regression")
print("=" * 60)

model4 = poisson(
    df=df_regress,
    outcome='count_events',
    inputs=['age', 'income'],
    robust=True
)

print("\nPoisson Regression Results:")
print(model4.get_tidy())

TEST 4: Poisson Regression
         Current function value: 149651730807018435982530780671770624000.000000
         Iterations: 35

Poisson Regression Results:
     term   estimate     std.error     statistic   p.value      conf.low  \
0   const -32.962559  1.658551e+07 -1.987431e-06  0.999998 -3.250703e+07   
1     age  -0.005334  1.996745e+05 -2.671477e-08  1.000000 -3.913549e+05   
2  income   0.000995  4.396441e+01  2.263666e-05  0.999982 -8.616766e+01   

      conf.high  
0  3.250696e+07  
1  3.913549e+05  
2  8.616965e+01  


In [7]:
# Test 5: Model Comparison
print("=" * 60)
print("TEST 5: Model Comparison")
print("=" * 60)

# Create two models with different specifications
model_simple = ols(df_regress, 'satisfaction', 'age')
model_full = ols(df_regress, 'satisfaction', ['age', 'income', 'education'])

comparison = compare_models(model_simple, model_full)
print("\nModel Comparison:")
print(comparison)

TEST 5: Model Comparison

Model Comparison:
     Model                  Inputs    N         AIC         BIC  \
0  Model 1                     age  200  601.672931  608.269566   
1  Model 2  age, income, education  200  552.580030  565.773299   

   Log-Likelihood  R_squared  Adj_R_squared  
0     -298.836465   0.142634       0.138304  
1     -272.290015   0.342529       0.332466  


In [8]:
# Test 6: Predictions
print("=" * 60)
print("TEST 6: Predictions")
print("=" * 60)

predictions = model1.predict()
print(f"\nPredicted values (first 10): {predictions[:10]}")

# New data predictions
new_data = pd.DataFrame({
    'age': [30, 40, 50],
    'income': [40000, 50000, 60000],
    'education': [16, 16, 18]
})
new_predictions = model1.predict(new_data)
print(f"\nPredictions for new data: {new_predictions}")

TEST 6: Predictions

Predicted values (first 10): 0    11.009774
1    10.799499
2    10.407709
3    10.768853
4    10.643722
5    10.332796
6    11.075364
7    11.604443
8     9.774625
9    11.151095
dtype: float64

Predictions for new data: 0    10.134068
1    10.557665
2    11.417297
dtype: float64


In [9]:
# Test 7: VIF (Multicollinearity Check)
print("=" * 60)
print("TEST 7: Variance Inflation Factors")
print("=" * 60)

vif_results = model1.vif()
print("\nVIF Values:")
print(vif_results)
print("\nNote: VIF > 10 indicates potential multicollinearity")

TEST 7: Variance Inflation Factors

VIF Values:
    Variable       VIF
1        age  1.012199
2     income  1.020473
3  education  1.013279

Note: VIF > 10 indicates potential multicollinearity


# Publication Tables

In [10]:
import importlib
from sociopathit.analyses import pubtable as pubtable_module
from IPython.display import HTML, display

importlib.reload(pubtable_module)
from sociopathit.analyses.pubtable import (
    proportion_table, 
    descriptive_table, 
    regression_table,
    save_table
)

# Simulate survey data
np.random.seed(42)
n = 300
df_survey = pd.DataFrame({
    'gender': np.random.choice(['Male', 'Female'], n),
    'education': np.random.choice(['High School', 'Bachelor', 'Graduate'], n),
    'response': np.random.choice(['Yes', 'No', 'Maybe'], n, p=[0.4, 0.4, 0.2]),
    'age': np.random.normal(40, 15, n),
    'income': np.random.normal(55000, 25000, n),
    'satisfaction': np.random.normal(7, 2, n),
    'weight': np.random.uniform(0.8, 1.2, n)
})

print(df_survey.head())
print(f"\nData shape: {df_survey.shape}")

   gender    education response        age         income  satisfaction  \
0    Male  High School       No  30.743644  106059.910568      7.746792   
1  Female  High School    Maybe  21.117369   67129.377631      8.014032   
2    Male     Graduate       No  57.832015   58632.405994      7.396579   
3    Male     Bachelor      Yes  39.380374   97839.349771      6.480788   
4    Male     Bachelor      Yes  42.609495   80253.424981      7.361291   

     weight  
0  1.190396  
1  1.066940  
2  0.804171  
3  0.902165  
4  0.904285  

Data shape: (300, 7)


In [11]:
# Test 1: Proportion Table (One-way)
print("=" * 60)
print("TEST 1: One-Way Proportion Table")
print("=" * 60)

html1 = proportion_table(
    df=df_survey,
    row_var='response',
    title='Distribution of Responses',
    ci=True,
    show_n=True
)

display(HTML(html1))

TEST 1: One-Way Proportion Table


response,%,95% CI,n
Maybe,18.0,"[10.0, 30.3]",54
No,42.0,"[33.7, 50.7]",126
Yes,40.0,"[31.7, 48.9]",120


In [12]:
# Test 2: Proportion Table (Cross-tabulation)
print("=" * 60)
print("TEST 2: Cross-Tabulation Proportion Table")
print("=" * 60)

html2 = proportion_table(
    df=df_survey,
    row_var='response',
    col_var='gender',
    title='Response by Gender',
    decimals=1
)

display(HTML(html2))

TEST 2: Cross-Tabulation Proportion Table


response,Female,Male
Maybe,20.8,15.2
No,38.9,45.0
Yes,40.3,39.7


In [13]:
# Test 3: Weighted Proportion Table
print("=" * 60)
print("TEST 3: Weighted Proportion Table")
print("=" * 60)

html3 = proportion_table(
    df=df_survey,
    row_var='education',
    weight_var='weight',
    title='Education Distribution (Weighted)',
    ci=True,
    show_n=True
)

display(HTML(html3))

TEST 3: Weighted Proportion Table


education,%,95% CI,n
Bachelor,35.1,"[26.6, 44.7]",104
Graduate,31.0,"[22.6, 41.0]",94
High School,33.9,"[25.4, 43.5]",102


In [14]:
# Test 4: Descriptive Statistics Table
print("=" * 60)
print("TEST 4: Descriptive Statistics Table")
print("=" * 60)

html4 = descriptive_table(
    df=df_survey,
    variables=['age', 'income', 'satisfaction'],
    stats=['mean', 'sd', 'min', 'max', 'n'],
    decimals=2,
    title='Descriptive Statistics',
    var_labels={'age': 'Age (years)', 'income': 'Annual Income ($)', 'satisfaction': 'Life Satisfaction'}
)

display(HTML(html4))

TEST 4: Descriptive Statistics Table


Variable,Mean,Sd,Min,Max,N
Age (years),41.73,16.11,-1.77,84.64,300
Annual Income ($),57483.38,26825.21,-10246.04,134184.29,300
Life Satisfaction,7.08,2.04,1.97,12.40,300


In [15]:
# Test 5: Grouped Descriptive Statistics
print("=" * 60)
print("TEST 5: Grouped Descriptive Statistics")
print("=" * 60)

html5 = descriptive_table(
    df=df_survey,
    variables=['age', 'income'],
    group_var='gender',
    stats=['mean', 'sd', 'n'],
    decimals=1,
    title='Demographics by Gender'
)

display(HTML(html5))

TEST 5: Grouped Descriptive Statistics


Variable,Group,Mean,Sd,N
age,Female,42.2,15.4,149
,Male,41.2,16.8,151
income,Female,57561.4,26690.6,149
,Male,57406.4,26957.1,151


In [16]:
# Test 6: Regression Table (Single Model)
print("=" * 60)
print("TEST 6: Regression Table (Single Model)")
print("=" * 60)

# Fit a model
model_pub = ols(df_survey, 'satisfaction', ['age', 'income'])
results_df = model_pub.get_tidy()

html6 = regression_table(
    models=results_df,
    title='OLS Regression: Life Satisfaction',
    show_se=True,
    show_stars=True,
    var_labels={'age': 'Age', 'income': 'Income', 'const': 'Intercept'},
    stats_rows={
        'N': [int(model_pub.get_stats()['N'])],
        'R_squared': [model_pub.get_stats()['R_squared']]
    }
)

display(HTML(html6))

TEST 6: Regression Table (Single Model)


,Model 1
Intercept,7.958***
,(0.406)
Age,-0.009
,(0.008)
Income,-0.000*
,(0.000)
N,300.00
R_squared,0.02


In [17]:
# Test 7: Regression Table (Multiple Models)
print("=" * 60)
print("TEST 7: Regression Table (Multiple Models)")
print("=" * 60)

# Fit multiple models
model_a = ols(df_survey, 'satisfaction', 'age')
model_b = ols(df_survey, 'satisfaction', ['age', 'income'])

html7 = regression_table(
    models=[model_a.get_tidy(), model_b.get_tidy()],
    model_names=['Model 1', 'Model 2'],
    title='Regression Models Comparison',
    show_se=True,
    show_stars=True,
    var_labels={'age': 'Age', 'income': 'Income', 'const': 'Intercept'},
    stats_rows={
        'N': [int(model_a.get_stats()['N']), int(model_b.get_stats()['N'])],
        'R_squared': [model_a.get_stats()['R_squared'], model_b.get_stats()['R_squared']]
    }
)

display(HTML(html7))

TEST 7: Regression Table (Multiple Models)


,Model 1,Model 2
Intercept,7.495***,7.958***
,(0.348),(0.406)
Age,-0.010,-0.009
,(0.008),(0.008)
Income,,-0.000*
,,(0.000)
N,300.00,300.00
R_squared,0.01,0.02


In [18]:
# Test 8: Regression Table with Confidence Intervals
print("=" * 60)
print("TEST 8: Regression Table with Confidence Intervals")
print("=" * 60)

html8 = regression_table(
    models=model_pub.get_tidy(),
    title='OLS Regression with Confidence Intervals',
    show_se=False,
    show_ci=True,
    show_stars=True,
    var_labels={'age': 'Age', 'income': 'Income', 'const': 'Intercept'}
)

display(HTML(html8))

TEST 8: Regression Table with Confidence Intervals


,Model 1
Intercept,7.958***
,"[7.162, 8.754]"
Age,-0.009
,"[-0.025, 0.006]"
Income,-0.000*
,"[-0.000, -0.000]"


# Summary

All analyses functions tested successfully:

## Regression Module (`regress.py`)
- OLS regression (unweighted and weighted)
- Logistic regression
- Poisson regression
- Model comparison
- Predictions
- VIF diagnostics

## Publication Tables Module (`pubtable.py`)
- Proportion tables (one-way and cross-tabulation)
- Weighted proportion tables
- Descriptive statistics tables
- Grouped descriptive statistics
- Regression coefficient tables (single and multiple models)
- Tables with confidence intervals and significance stars